In [25]:
# packages
import os
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [3]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
df = pd.read_csv('/content/drive/My Drive/ms_wind_curtailment_prediction/curtailment_target_features.csv', sep = ';', index_col=0)

In [23]:
# Split the entire dataset into training and test sets
cutoff_time = "2023-01-01"
train = df[df.index < cutoff_time]
test = df[df.index >= cutoff_time]
X_train = train.drop('redispatch', axis = 1)
y_train = train['redispatch']
X_test = test.drop('redispatch', axis = 1)
y_test = test['redispatch']

redispatch                           0
wind_speed_m/s                      16
wind_direction_degrees              16
radiation_global_J/m2               16
air_temperature_K                   16
humidity_percent                    16
wind_gust_max_m/s                   16
wind_direction_gust_max_degrees     16
forecast_solar_MW                  112
actual_solar_MW                    197
total_grid_load_MWh                 16
residual_load_MWh                   16
pumped_storage_MWh                  16
dtype: int64

In [ ]:
# Define the preprocessing steps
imputer = SimpleImputer(strategy='constant', fill_value=0)  # Replace missing values with 0
scaler = StandardScaler()
smote = SMOTE()
# Create the preprocessing pipeline
preprocessor = Pipeline([
    ('imputer', imputer),
    ('scaler', scaler),
    ('smote', smote)
])

# Example usage:
# Fit the pipeline on training data and transform it
X_train_preprocessed = preprocessor.fit_transform(X_train, y_train)

# Transform the test data using the fitted pipeline
X_test_preprocessed = preprocessor.transform(X_test)

In [ ]:
# Define dictionary to store results
training_history = {}

# Define number of epochs and learning rate decay
N_TRAIN = len(X_train)
EPOCHS = 2000
BATCH_SIZE = 32
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.01,
    decay_steps=STEPS_PER_EPOCH*1000,
    decay_rate=1,
    staircase=False)

# Define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, name='Adam')

# define model
def get_compiled_large_model():
    large_model = tf.keras.Sequential([
      tf.keras.layers.Dense(512,kernel_initializer = 'uniform', activation='relu', input_dim = 12),
      tf.keras.layers.Dense(512,kernel_initializer = 'uniform', activation='relu'),
      tf.keras.layers.Dense(512,kernel_initializer = 'uniform', activation='relu'),
      tf.keras.layers.Dense(512,kernel_initializer = 'uniform', activation='relu'),
      tf.keras.layers.Dense(1,kernel_initializer = 'uniform')
    ])

    large_model.compile(optimizer=optimizer,
                  loss='mae',
                  metrics=['mse'])
    return large_model

# Summary of large sized model
with tf.device('/cpu:0'):
    large_model = get_compiled_large_model()
    print(large_model.summary())

# train
# Train the model with the new callback
with tf.device('/cpu:0'):
    training_history['large'] = large_model.fit(X_train,
                        y_train,
                        validation_split=0.2,
                        verbose=0,
                        steps_per_epoch=STEPS_PER_EPOCH,
                        epochs=EPOCHS)